In [4]:
import pandas as pd
df_trade = pd.read_parquet("/home/ubuntu/temp/trade_updates.parquet")
df_order = pd.read_parquet("/home/ubuntu/temp/order_updates.parquet")

ArrowInvalid: Could not open Parquet input source '<Buffer>': Parquet magic bytes not found in footer. Either the file is corrupted or this is not a parquet file.

In [27]:
# fn extract_strategy_id(order_id: i64) -> i32 {
#     (order_id >> 32) as i32
# }
df_trade["strategy_id"] = df_trade["client_order_id"].apply(lambda x: (x >> 32) & 0xFFFFFFFF)
df_order["strategy_id"] = df_order["client_order_id"].apply(lambda x: (x >> 32) & 0xFFFFFFFF)

In [3]:
df_open_sig = pd.read_parquet("/home/ubuntu/temp/arb_open_signals.parquet")

In [35]:
df_order["strategy_id"].head(3)

0    121123390
1    126288409
2    131348826
Name: strategy_id, dtype: int64

In [36]:
df_open_sig[df_open_sig["strategy_id"] == 121123390]

,key,ts_us,strategy_id,create_ts,opening_venue,opening_symbol,opening_bid0,opening_ask0,hedging_venue,hedging_symbol,...,side,order_type,price,price_tick,exp_time,open_threshold,hedge_timeout_us,funding_ma,predicted_funding_rate,loan_rate
84,00001763683344069121_0121123390,1763683344069121,121123390,1763683344069121,BinanceMargin,SAGAUSDT,0.1107,0.1108,BinanceUm,SAGAUSDT,...,SELL,LIMIT,0.1108,0.0001,1763683464069121,0.0,30000000,-0.001397,0.000029,0.000294


In [37]:
df_open_sig[df_open_sig["strategy_id"] == 126288409]

,key,ts_us,strategy_id,create_ts,opening_venue,opening_symbol,opening_bid0,opening_ask0,hedging_venue,hedging_symbol,...,side,order_type,price,price_tick,exp_time,open_threshold,hedge_timeout_us,funding_ma,predicted_funding_rate,loan_rate
83,00001763683349234036_0126288409,1763683349234036,126288409,1763683349234036,BinanceMargin,SAGAUSDT,0.1105,0.1106,BinanceUm,SAGAUSDT,...,SELL,LIMIT,0.1106,0.0001,1763683469234036,0.0,30000000,-0.001397,0.000029,0.000294


In [38]:
df_trade[df_trade["strategy_id"] == 126288409]

,key,ts_us,event_time,trade_time,symbol,trade_id,order_id,client_order_id,side,price,quantity,commission,commission_asset,is_maker,realized_pnl,trading_venue,cumulative_filled_quantity,order_status,strategy_id
0,00001763683432748200,1763683432748200,1763683432746,1763683432739,SAGAUSDT,260821944,4212896749,542404586518872066,BUY,0.1100,904.2,0.009946,USDT,True,0.396129,BinanceUm,904.2,FILLED,126288409
4,00001763683416500263,1763683416500263,1763683416487,1763683416486,SAGAUSDT,77375209,1949101673,542404586518872065,SELL,0.1106,904.2,0.040002,USDT,True,0.000000,BinanceMargin,904.2,FILLED,126288409


In [ ]:
import pandas as pd
df_trade = pd.read_parquet("/home/ubuntu/temp/arb_open_signals.parquet")